## Generate Hyp3 interferogram stack

This notebook will queue up processing jobs in Hyp3 and 
then download and prepare them for use in Mintpy. 
Code adapted from [Hyp3 tutorial](https://nbviewer.org/github/ASFHyP3/hyp3-docs/blob/main/docs/tutorials/hyp3_insar_stack_for_ts_analysis.ipynb).

### Prepare environment

Import functions and load metadata for processing. The environment requires several packages from the Alaska Satellite Facility to function, specifically [asf_search](https://docs.asf.alaska.edu/asf_search/basics/) and [hyp3_sdk](https://hyp3-docs.asf.alaska.edu/using/sdk/).

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
from dateutil.parser import parse as parse_date
from typing import List, Union
from osgeo import gdal
from solid_utils import permafrost_utils as pu
import asf_search as asf
import hyp3_sdk as sdk
import json

In [ ]:
################# Set Directories, get site info ##############################
site = 'NorthSlopeEastD102'
requirement = 'permafrost'
dataset = 'Hyp3_S1'
year = 2025
start_directory = 'default'

custom_sites = "/home/jovyan/my_sites.txt"  # Path to custom site metadata
try:
    with open(custom_sites, "r") as f:
        sitedata = json.load(f)
    site_info = sitedata["sites"][site]
except (FileNotFoundError, json.JSONDecodeError) as e:
    raise RuntimeError(f"Failed to load site metadata from {custom_sites}: {e}")
except KeyError:
    raise ValueError(f"Site ID '{site}' not found in {custom_sites}")

scratch_path = "/scratch/nisar-st-calval-solidearth/permafrost/Hyp3_S1"
work_dir = Path(f'{scratch_path}/{site}/{str(year)}')
print("Work directory:", work_dir)
work_dir.mkdir(parents=True, exist_ok=True)
# Change to Workdir   
os.chdir(work_dir)
       
hyp3_dir = work_dir/'products'               #aka gunwdir
hyp3_dir.mkdir(parents=True, exist_ok=True)
print("   Hyp3  dir:", hyp3_dir) 
    
mintpy_dir = work_dir/'MintPy' 
mintpy_dir.mkdir(parents=True, exist_ok=True)
print("   MintPy  dir:", mintpy_dir)

### Generate SBAS image pairs

Identify stack of SLC images and determine SBAS pairs.

In [ ]:
lonlat = site_info['region_identifier']
frame = site_info['sentinel_frame']
if site_info['sentinel_direction']=='ASCENDING':
    direction = asf.constants.ASCENDING
elif site_info['sentinel_direction']=='DESCENDING':
    direction = asf.constants.DESCENDING

stack_start = parse_date(f'{year}-05-20 00:00:00Z')
stack_end = parse_date(f'{year}-09-20 00:00:00Z')

search_results = asf.geo_search(platform=asf.constants.SENTINEL1, 
                                intersectsWith=lonlat, 
                                start=stack_start,
                                end=stack_end,
                                processingLevel=asf.constants.SLC,
                                beamMode=asf.constants.IW,
                                flightDirection=direction,
                                asfFrame=int(frame))

In [ ]:
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] 
        for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

stack

In [ ]:
sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(
    index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference)
        & (stack.temporalBaseline - rt <= int(site_info['tempBaseMax']))
        & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

### Do Hyp3 Processing

This step requires NASA Earthdata credentials. See [Hyp3 documentation](https://hyp3-docs.asf.alaska.edu/) for more information about using Hyp3 processing. Each user has an allotted Hyp3 processing quota, for which some of this processing is counted against.

In [ ]:

hyp3 = sdk.HyP3(prompt=True)


In [ ]:

jobs = sdk.Batch()
h3projname = f"{site}_NCV_{year}"
for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=h3projname,
                                  include_dem=True, include_look_vectors=True)


This processing will likely take about an hour to complete. You cannot download data until the processing is finished. The following cell will check to see if the processing is still running.

In [ ]:
running = len(hyp3.find_jobs(status_code=["RUNNING"]))
pending = len(hyp3.find_jobs(status_code=["PENDING"]))

if running+pending ==0:
    print('No jobs running, all clear')
else:
    print(f'Jobs still running!\n{pending} pending, {running} running')

### Download Data


h3projname = f"{site}_NCV_{year}"
batch = hyp3.find_jobs(name=h3projname)
insar_products = batch.download_files(hyp3_dir)
insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]


### Crop Interferograms

In [ ]:
# Crop to common overlap
files = hyp3_dir.glob('*/*_dem.tif')
overlap = pu.get_common_overlap(files)
pu.clip_hyp3_products_to_common_overlap(hyp3_dir, overlap)

## Generate MintPy Config file

In [ ]:
def writenumpyconfig(config_file,data_dir,reflalo = None, subset = None):
    """Write a mintpy config file.
    config_file: file name to write to
    data_dir: directory of .tif files
    reflalo (optional): 'y,x' of reference point in tif proj
    subset (optional): '[ymin:ymax,xmin:xmax]' in tif proj
    
    Note: hyp3 provides images in a utm projection, so 
    lalo here refers to y,x in map coords"""
    
    cfgtext = f"""
    mintpy.load.processor        = hyp3
    ##---------interferogram datasets:
    mintpy.load.unwFile          = {data_dir}/*/*_unw_phase_clipped.tif
    mintpy.load.corFile          = {data_dir}/*/*_corr_clipped.tif
    ##---------geometry datasets:
    mintpy.load.demFile          = {data_dir}/*/*_dem_clipped.tif
    mintpy.load.incAngleFile     = {data_dir}/*/*_lv_theta_clipped.tif
    mintpy.load.azAngleFile      = {data_dir}/*/*_lv_phi_clipped.tif
    mintpy.load.waterMaskFile    = {data_dir}/*/*_water_mask_clipped.tif"""

    if reflalo:
            cfgtext+=f"""    
    mintpy.reference.lalo         = {reflalo}"""  #should be 'y,x' in map coords
    
    if not subset:
        subset = 'no'
    cfgtext+=f"""
    mintpy.subset.lalo            = {subset}""" #should be '[ymin:ymax,xmin:xmax]'
                                                #in map coords
        
    mintpy_config = config_file
    mintpy_config.write_text(cfgtext)

In [ ]:
config_file = mintpy_dir/f"{site_info['calval_location']}_{str(year)}.cfg"
writenumpyconfig(config_file,
                 hyp3_dir,
                 reflalo=site_info['mintpy_ref_loc'],
                 subset = site_info['subset_region'])